# Simulating Single Phase Transport 

The point of an OpenPNM simulation is ultimately to compute some transport process.  In this notebook we will cover the following subjects:

- Defining conductance 
- Settings boundary conditions 


Start by defining a network.  We'll use the ``Demo`` class which happens to include all the geometrical pore-scale models already. 

In [2]:
import numpy as np
import openpnm as op
op.visualization.set_mpl_style()

pn = op.network.Demo(shape=[5, 5, 1], spacing=5e-5)
print(pn)


══════════════════════════════════════════════════════════════════════════════
net : <openpnm.network.Demo at 0x23503894050>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  2  pore.coords                                                       25 / 25
  3  throat.conns                                                      40 / 40
  4  pore.coordination_number                                          25 / 25
  5  pore.max_size                                                     25 / 25
  6  throat.spacing                                                    40 / 40
  7  pore.seed                                                         25 / 25
  8  pore.diameter                                                     25 / 25
  9  throat.max_size                                                   40 / 40
 10  

## Define Phase Viscosity

To fully illustrate the process of performing transport calculations, we'll use an empty ``Phase`` object and add all the needed properties manually:

In [3]:
water = op.phase.Phase(network=pn)

Let's assume that we are interested in pressure driven flow. This requires knowing the viscosity of the phase, so let's add a pore-scale model for computing the viscosity of water:

In [4]:
water.add_model(propname='pore.viscosity',
                model=op.models.phase.viscosity.water_correlation)
print(water)


══════════════════════════════════════════════════════════════════════════════
phase_01 : <openpnm.phase.Phase at 0x23518f84c80>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  2  pore.temperature                                                  25 / 25
  3  pore.pressure                                                     25 / 25
  4  pore.viscosity                                                    25 / 25
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Labels                                                 Assigned Locations
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  2  pore.all                                                               25
  3  throat.all                                                             40
―

And we can check the individual values to verify they make sense:

In [5]:
print(water['pore.viscosity'])

[0.00089319 0.00089319 0.00089319 0.00089319 0.00089319 0.00089319
 0.00089319 0.00089319 0.00089319 0.00089319 0.00089319 0.00089319
 0.00089319 0.00089319 0.00089319 0.00089319 0.00089319 0.00089319
 0.00089319 0.00089319 0.00089319 0.00089319 0.00089319 0.00089319
 0.00089319]


## Basic Conductance Calculation

Determining the conductance of the conduits between each pair of connected pores is the most important part of performing a simulation. The details of conductance models are covered elsewhere. For this demonstration will assume the very simplest case where all pressure loss occurs in the throats.

Recall the Hagan-Poiseuille equation for fluid flow through a cylindrical tube:

$$ Q = \frac{\pi R^4}{8 \mu L} \Delta P$$

where $R$ and $L$ are the radius and length of the throat, and $\mu$ is the viscosity of the fluid.  Together this prefactor can be referred to as the hydraulic conductance, $g_h$, giving:

$$ Q = g_h \Delta P $$

So the aim is the compute values of $g_h$ for each throat.  We start by doing this manually:

In [6]:
R = pn['throat.diameter']/2
L = pn['throat.length']
mu = water['throat.viscosity']  # See ProTip below
water['throat.hydraulic_conductance'] = np.pi*R**4/(8*mu*L)
print(water['throat.hydraulic_conductance'])

[1.71474078e-15 4.89487491e-16 6.50123109e-16 2.09291190e-14
 5.09786041e-14 8.92694901e-15 9.28972043e-15 2.08610930e-14
 1.28181288e-15 1.48187324e-15 2.78733145e-14 7.76058006e-14
 1.04922730e-15 1.73610610e-14 2.61741594e-14 7.67328653e-16
 6.60091981e-14 6.40072101e-15 5.78941040e-15 3.29178076e-14
 2.14954939e-15 2.73094186e-15 5.22858424e-16 9.95589594e-14
 1.68107574e-14 3.53246965e-15 1.65944003e-15 7.86513283e-15
 7.64403364e-14 2.11288391e-14 9.50956907e-16 1.41678374e-15
 2.92086263e-14 2.49925306e-14 9.04335289e-16 1.19842395e-15
 1.68312420e-14 6.58546574e-15 2.25965139e-14 8.24975878e-16]


```{admonition} Phase can do automatic interpolation to get throat values
  The ``Phase`` class has a special ability to interpolate pore values to throats, and vice-versa. In PNM simulations all the balances are solved for each pore, so the thermodynamic properties like temperature, pressure, etc. are all defined on pores. Consequently, the physical properties are also defined in pores, like viscosity. However, as shown above we often want viscosity values in the throats. OpenPNM provides a shortcut for this, such that if you request a throat property that does not exist, it will attempt to fetch the pores values and do a linear interpolation of values to produce an array of throat values.  There is also a function for this, ``water.interpolate_data('throat.viscosity')``, but the ``water['throat.viscosity']`` shortcut is very convenient. The automatic interpolation can be disabled in the `phase.settings`.
```

In [7]:
water.interpolate_data('throat.viscosity')

array([0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319])

In [8]:
water['throat.viscosity']

array([0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319,
       0.00089319, 0.00089319, 0.00089319, 0.00089319, 0.00089319])

## Create Algorithm Object

OpenPNM contains a variety of ``Algorithm`` classes in the ``openpnm.algorithms`` module.  Let's initialize a ``StokesFlow`` algorithm, since this simulates pressure driven flow through the network. 

In [9]:
sf = op.algorithms.StokesFlow(network=pn, phase=water)
print(sf)


══════════════════════════════════════════════════════════════════════════════
stokes_01 : <openpnm.algorithms.StokesFlow at 0x23518f859a0>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  2  pore.bc.rate                                                       0 / 25
  3  pore.bc.value                                                      0 / 25
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Labels                                                 Assigned Locations
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  2  pore.all                                                               25
  3  throat.all                                                             40
―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

## Assign Boundary Conditions

As can be seen in the print-out above there are predefined ``'pore.bc'`` arrays, but they contain no valid values, meaning they are all ``nans``. Once we set some boundary conditions, this will change.  Let's apply pressure BCs on one side of the network, and rate BCs on the other:

In [10]:
sf.set_value_BC(pores=pn.pores('left'), values=100_000)
sf.set_rate_BC(pores=pn.pores('right'), rates=1e-10)
print(sf)


══════════════════════════════════════════════════════════════════════════════
stokes_01 : <openpnm.algorithms.StokesFlow at 0x23518f859a0>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  2  pore.bc.rate                                                       5 / 25
  3  pore.bc.value                                                      5 / 25
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Labels                                                 Assigned Locations
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  2  pore.all                                                               25
  3  throat.all                                                             40
―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

```{admonition} All boundary conditions are preceeded with 'pore.bc'
  All boundary conditions are stored as 'pore.bc.<type>', which means that OpenPNM's dictionary lookup tricks can be used to see all types and values of bc's using: `sf['pore.bc']` which will return a `dict`. This can be used as shown below:


In [11]:
print(sf['pore.bc'].keys())

dict_keys(['rate', 'value'])


Now we can see there are 5 valid values of each type.  The ``sf`` algorithm will look for ``'throat.hydraulic_conductance'`` on ``water`` by default, so we can just ``run``:

In [12]:
soln = sf.run()

The ``run`` method solves the mass balance around each pore and computes the pressure within each pore that is required to sustain the flow defined by the boundary conditions.  ``soln`` return None, instead the ``'pore.pressure'`` values are written to the dictionary of the algorithm object:

In [13]:
print(sf)


══════════════════════════════════════════════════════════════════════════════
stokes_01 : <openpnm.algorithms.StokesFlow at 0x23518f859a0>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  2  pore.bc.rate                                                       5 / 25
  3  pore.bc.value                                                      5 / 25
  4  pore.pressure                                                     25 / 25
  5  pore.initial_guess                                                25 / 25
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Labels                                                 Assigned Locations
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  2  pore.all                                                        

Finally we can look at how much pressure was required in the "right" pores to meet the required flow rate:

In [14]:
sf['pore.pressure'][pn.pores('right')]

array([139569.47740193, 138265.16063556, 134437.32490449, 128801.56476401,
       131525.91604191])

So we can see that 150 kPa was required to accomplish the requested flow. 

## Rigorous Conductance Calculation

The above demonstration used a very simplistic conductance calculation.  It was also stated that computing conductance is the most important part of doing a PNM simulation.  To finish this notebook, we'll look more closely at this process.

### Manual Method
Let's print the network object again:

In [15]:
print(pn)


══════════════════════════════════════════════════════════════════════════════
net : <openpnm.network.Demo at 0x23503894050>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  2  pore.coords                                                       25 / 25
  3  throat.conns                                                      40 / 40
  4  pore.coordination_number                                          25 / 25
  5  pore.max_size                                                     25 / 25
  6  throat.spacing                                                    40 / 40
  7  pore.seed                                                         25 / 25
  8  pore.diameter                                                     25 / 25
  9  throat.max_size                                                   40 / 40
 10  

Note the ``'throat.hydraulic_size_factors'`` array.  This is computed by a pore-scale model on the ``Demo`` network.  This computation is more rigorous in the following ways:

1. The conductance of each half pore and that throat is considered.
1. The throat length is computed carefully by accounting for the 'lens' between the intersection of the spherical pore bodies and the cylindrical throat.
1. The net cross-sectional area of the pores are computed by integrating between the pore center and the pore-throat intersection point

The conductance of each element in the conduit is returned as an *Nt-by-3* array, where columns 1 and 3 contain the hydraulic conductance of the half pore on either end of the throat, and the column 1 contains the throat conductance.

In [16]:
pn['throat.hydraulic_size_factors']

array([[5.53646621e-17, 1.53159095e-18, 5.80704541e-17],
       [3.27057594e-17, 4.37205796e-19, 2.21236821e-17],
       [2.21236821e-17, 5.80684076e-19, 8.15497372e-17],
       [3.86668667e-16, 1.86936997e-17, 2.61233255e-16],
       [4.92226337e-16, 4.55336277e-17, 5.49581815e-16],
       [2.58445082e-16, 7.97347004e-18, 1.46626967e-16],
       [1.46626967e-16, 8.29749419e-18, 2.76612840e-16],
       [3.92929505e-16, 1.86329395e-17, 2.57815992e-16],
       [5.47578209e-17, 1.14490366e-18, 4.42265276e-17],
       [4.42265276e-17, 1.32359576e-18, 9.36797895e-17],
       [3.28052595e-16, 2.48961922e-17, 4.16483115e-16],
       [6.26881484e-16, 6.93167986e-17, 6.97015739e-16],
       [3.55626308e-17, 9.37160330e-19, 7.14255758e-17],
       [2.28811833e-16, 1.55067425e-17, 3.58288320e-16],
       [4.27538842e-16, 2.33785222e-17, 3.04140609e-16],
       [6.77654132e-17, 6.85371006e-19, 2.76338664e-17],
       [5.87496107e-16, 5.89588182e-17, 6.14445225e-16],
       [2.23127657e-16, 5.71706

This data is called the ``size factor`` because it is purely the geometrical information required for the computation of the hydraulic conductance.  So the Hagan-Poisseiulle equation for each element is written as:

$$ Q = \frac{F_h}{\mu} \Delta P = g_h \Delta P$$

Note that both the $\frac{\pi R^4}{8}$ term and $L$ have been rolled into the $F_h$ value.

The total conductance of the pore-throat-pore conduit can be found as the sum of three resistors in series.  Since we have conductance values, we add the inverses, and invert again.  The full expression for the hydraulic conductance between pores i and j, through throat k, is:


$$ Q = \bigg( \frac{\mu}{F_{h, i}} + \frac{\mu}{F_{h, k}} + \frac{\mu}{F_{h, j}} \bigg) ^ {-1} \Delta P $$

This can be computed by hand:

In [17]:
F_h = water['throat.hydraulic_size_factors']
water['throat.hydraulic_conductance'] = (mu * (1/F_h).sum(axis=1))**(-1)

In [18]:
water['throat.hydraulic_conductance']

array([1.62682953e-15, 4.73790919e-16, 6.29130422e-16, 1.86882996e-14,
       4.33728726e-14, 8.22585155e-15, 8.54945776e-15, 1.86310861e-14,
       1.22451096e-15, 1.41934189e-15, 2.45435403e-14, 6.41358604e-14,
       1.00938371e-15, 1.56258033e-14, 2.31312623e-14, 7.41440712e-16,
       5.51773052e-14, 5.95456230e-15, 5.39441483e-15, 2.87003409e-14,
       2.04722414e-15, 2.58964133e-15, 5.06889705e-16, 8.06300420e-14,
       1.50606027e-14, 3.33244184e-15, 1.58693450e-15, 7.25921434e-15,
       6.32419568e-14, 1.88596095e-14, 9.12930789e-16, 1.35697434e-15,
       2.56723985e-14, 2.21254255e-14, 8.71671454e-16, 1.15158254e-15,
       1.51653544e-14, 6.12147269e-15, 2.00401599e-14, 7.96548498e-16])

In [19]:
sf = op.algorithms.StokesFlow(network=pn, phase=water)
sf.set_value_BC(pores=pn.pores('left'), values=100_000)
sf.set_rate_BC(pores=pn.pores('right'), rates=1e-10)
soln = sf.run()
sf['pore.pressure'][pn.pores('right')]

array([144533.4827636 , 142990.61847937, 138772.80703712, 132858.77887604,
       135952.53964271])

As can be seen the numbers are about the same as with the simple case, but should be somewhat more correct.  In fact, these above pressures are a bit higher, which is because the total conductance of the conduit is lower due to the inclusion of the pore body lengths into the total length, compared to above where only the throat length was included.

### Pore-Scale Model Method

Instead of computing the hydraulic conductance manually as done above, there is a pore-scale model available:

In [20]:
water.add_model(propname='throat.hydraulic_conductance',
                model=op.models.physics.hydraulic_conductance.generic_hydraulic)

In [21]:
sf = op.algorithms.StokesFlow(network=pn, phase=water)
sf.set_value_BC(pores=pn.pores('left'), values=100_000)
sf.set_rate_BC(pores=pn.pores('right'), rates=1e-10)
soln = sf.run()
sf['pore.pressure'][pn.pores('right')]

array([144533.4827636 , 142990.61847937, 138772.80703712, 132858.77887604,
       135952.53964271])

Which gives exactly the same result, without have to manually deal with the conductances-in-series calculation.